---
# 삼성 INTENSIVE 현업 PROJECT
---

---
## **기획**


### [배경 설명 & 요구사항 확인]

![플라즈마 이미지](image/project/plasma.jpg)

> ### 배경
- 플라즈마 사용 공정 설비에서 문제 발생할 경우, 플라즈마 광 신호 세기가 불안정해집니다.
- 제가 하는 업무는 이러한 플라즈마 이상 신호를 진단하기 위해, 플라즈마 광 신호를 실시간 측정하는 센서를 개발하고 있습니다.

  ※ 플라즈마에서 방출된 photon을 electron으로 변환하여, PMT 소자를 이용해 플라즈마 광 신호 세기를 측정하는 원리입니다.

> ### 요구사항
- 플라즈마 광 신호 데이터 양이 너무 많아 분석하는데 시간이 오래 걸립니다. (50,000 points/s)
- 플라즈마 이상 신호를 자동으로 진단하는 모델을 개발해서 이상 신호를 조기에 감지하는 것을 목표로 합니다.
- target 이상 신호는 1 point 튀는 outlier 신호와 불안정하게 진동하는 수십 points 신호입니다.
- 현재 과정에서의 목표는 설비 별로 발생하는 이상 신호를 진단하는 개별 모델을 만드는 것입니다.
- 최종적으로 엔지니어에게 알람을 제공하는 시스템 구축해, 플라즈마 공정 설비 안정성을 유지하는 것을 목표로 합니다.

### [문제 범위 설정]

- 주요 목표는 플라즈마 광 신호를 실시간 측정하는 센서로부터 생성된 데이터를 사용하여 이상 신호 진단하는 모델을 개발하는 것입니다.

### [문제 정의 도출]

- 센서 데이터 기반으로 이상 신호를 자동으로 진단하는 자동 이상 진단 모델을 개발합니다.
  1. 플라즈마 광 신호의 정상 범위 정의
  2. 정상 범위를 벗어나는 이상 신호 정의
  3. 정상 신호와 이상 신호 식별하는 모델 개발
  4. 개발 모델 검증, 성능 평가 지표 설정
  5. 테스트 후 모델 조정 및 개선

### [데이터 확보 방법]

- 센서 데이터 사외 반출이 안 되어 직접 제작했습니다.
- 실제 신호 데이터와 최대한 유사하게 base signal을 만들고 noise와 이상 신호를 추가하여 직접 만들었습니다.

   ※ 실제 현업에서 사용하는 데이터의 양이 너무 많아 도저히 만들 수 없어 최대한 많은 포인트로 제작했습니다. (avg 1,000 points/signal)

### [활용 데이터 정의]

- 데이터 소스: 플라즈마 광 센서 데이터
- 데이터 유형:
   1. 정형 데이터이며, 2차원 배열 형식의 숫자 데이터
- Feature:
   1. Feature는 센서 데이터의 인덱스 (시간 순, 마지막 column 전까지)
   2. 마지막 column은 target column
   3. target column의 값은 0(정상), 1, 2, 3, 4, 5, 6 (이상치/흔들림, 구간별)
- 형태:
   1. pickle 파일
   2. 총 4가지의 다른 유형의 신호 데이터를 가짐
   3. 각 파일은 13,600 건의 레코드를 포함
   4. column 개수는 신호 길이에 따라 다름

### [분석시 예상되는 어려움]

- 고차원 데이터: 데이터 양이 많아 리소스가 많이 투입되고 전처리 하기 어려움 (1 파일의 shape: 13,600 x 1,000, 4개의 파일)
- 이상치 데이터 불균형: 정상 신호 1,000 개 중 이상치 1개 (0.1% 이하)
- 클래스 불균형: 정상 신호와 이상 신호간 비율이 불균형 (10,000 vs 3,600)
- 센서 노이즈 및 오차: 빛의 신호가 약해질 수록 상대적으로 노이즈가 커져 모델 성능 저하 가능성 존재
- 모델 설명 가능성: 상사를 설득해야 하는 업무 특성 상, 딥러닝 같이 설명 불가능한 모델은 사용이 어려움

### [문제 해결을 위한 모델 사용 방법]

- 데이터 전처리
   1. 데이터 전처리
      - 결측치가 존재하는 경우 해당 행은 제거
      - 모두 동일한 값을 가지는 column 제거 (특정 feature가 모든 샘플에 같은 상수 값을 가짐)
      - column끼리 비교하여 같은 값을 가지는 column들이 있다면 제거

- 데이터 시각화
   1. 데이터 탐색
      - 데이터 구조를 파악하여 특징 확인 (head, columns, unique, describe, info)
      - 이상치나 노이즈를 확인하기 위해 데이터 분포 시각화 및 기초 통계량 확인
      - target 값 별로 어떤 종류의 이상 신호인지 확인
      - 각 특성 분포를 시각화하여 데이터 특성 파악
      - 정상 신호와 이상 신호 간의 클래스 비율 확인하여 불균형 있는지 확인
      - 신호 한 개의 데이터에서 이상 신호 데이터 비율 확인 (비정상/정상)

- 모델 선택과 학습
   1. Feature Scaling
      - Standardization
   2. 모델 선택
      - 설명 불가능한 딥러닝 모델 보다는 설명할 수 있는 머신러닝 모델을 우선 고려
      - target 데이터가 있기 때문에 분류해주는 머신러닝 지도학습 모델 평가 (강사님이 다 돌려보래서 배운 분류 모델 전부 돌릴 예정)
         1. KNN
         2. Logistic Regression
         3. Naive Bayes
         4. Decision Tree
         5. Support Vector Machine
         6. Random Forest
         7. AdaBoost
         8. GradientBoosting
         9. LDA
      - 위에서 나온 결과를 토대로 앙상블 모델도 평가
         1. Voting
         2. Bagging
         3. Stacking
      - 파일 용량이 커, 학습 시간이 오래 걸리면 PCA나 LDA를 이용해 차원 축소 후 모델 적용
   3. 모델 학습
      - 데이터를 학습, 검증, 테스트 셋으로 나누어서 학습

- 모델 평가와 해석
   1. 성능 평가
      - 모델의 성능을 평가하기 위해 accuracy, prcision, recall, f1 score, confusion matrix 등을 사용
   2. 모델 해석
      - 모델의 의사 결정 과정을 해석하기 위해 모델의 특성 중요도를 확인하고, SHAP(SHapley Additive exPlanations) 등의 기법을 사용하여 모델의 동작을 설명

- 모델 최적화와 향상
   1. 모델 개선
      - Cross validation을 이용하여 모델 평가
      - GridSearchCV를 이용하여 모델 별 하이퍼 파라미터 튜닝을 통한 모델 성능 최적화
      - 불필요한 특성 탐색 후, 제거
      - 앙상블 기법 사용해서 최적화 모델 생성

---
### **내용이 너무 길어지는 관계로 총 4개의 파일 중 첫 번째 파일에 대한 전체 과정을 진행한 후**

### **나머지 세 개 파일에 대해서는 모델 평가 결과만 공유드리겠습니다.**

---
## **준비**

### [데이터 수집 및 로드]

- 사내 센서 저장소에서 신호 데이터를 추출합니다.
- 데이터는 tdms 형식으로 저장되며, python tdms 라이브러리를 사용해서 로드합니다.

   ※ tdms는 National Instrumnet 독자 포맷으로 적용이 어려워, 여기서는 편의상 빠른 속도의 pickle을 사용하겠습니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# df_signal1 = pd.read_pickle('./data/signal/data_1.pkl')
# df_signal2 = pd.read_pickle('./data/signal/data_2.pkl')
# df_signal3 = pd.read_pickle('./data/signal/data_3.pkl')
# df_signal5 = pd.read_pickle('./data/signal/data_5.pkl')

df = pd.read_pickle('./data/signal/data_1.pkl')

### [데이터 전처리]

---
결측치 확인 및 확인 시, 해당 row 제거

In [ ]:
def report_null_rows(df):
    null_rows = df.isnull().sum(axis=1)
    rows_with_null = null_rows[null_rows != 0].index.tolist()
    print('제거할 행:', rows_with_null)
    return df.drop(index=rows_with_null)

df = report_null_rows(df)

---
모두 같은 상수 값을 가지는 column 제거

In [ ]:
def remove_uniform_columns(df):
    columns_to_remove = [col for col in df.columns if df[col].nunique() == 1]
    print('제거된 열:', columns_to_remove)
    return df.drop(columns=columns_to_remove)

df = remove_uniform_columns(df)

---
column끼리 비교하여 같은 값을 가지는 column들이 있다면 제거

In [ ]:
def check_duplicate_columns(df):
    df_T = df.T.drop_duplicates().T
    columns_to_remove = df.columns.difference(df_T.columns)
    df_T['target'] = df_T['target'].astype(int)
    print('제거된 열:', columns_to_remove.tolist())
    return df_T

df = check_duplicate_columns(df)

데이터 전처리 과정을 통해 아래 내용을 파악했습니다.
- isnull().sum() 이용, 해당 데이터에 결측치가 없음을 확인했습니다.
- nunique()==1 이용, 값이 전부 동일한 상수인 column이 없음을 확인했습니다. (target과 관련 없는 무의미한 feature)
- drop_duplicates() 이용, 같은 값을 가지는 column이 없습니다. (중복되어 선형 족속인 feature)

<br>

가이드에는 데이터 전처리 시, 데이터 스케일링 하라고 적혀져 있지만,

**저는 데이터 스케일링은 EDA를 통해 데이터 특성 파악 및 분석 후, 진행하겠습니다.**

---
## **EDA**

### <EDA 가이드>
- 데이터의 구조, 패턴, 이상치, 상관 관계 등을 파악
- 시각화 도구를 사용하여 데이터를 분석
- 모델의 성능을 향상시키기 위해 피처를 선택, 변환, 생성
- 데이터의 정보를 최대한 활용

In [ ]:
df.head()

2차원 배열 형식의 숫자형 데이터이며, 마지막 column은 target

---

In [ ]:
df.columns

column은 0 ~ 971까지 순차적으로 증가하다(feature) 마지막 column은 target

---

In [ ]:
df.info()

df의 행은 13,600개, 열은 973개
feature: 972개 열이며 float64 type
target: 1개의 열이며 int32 type

---

In [ ]:
df.describe()

feature를 봤을 때 (앞, 뒤쪽만)
1. 표준편차는 크게 변하지 않음
2. feature index가 커질 수록, std 제외 다른 기초 통계량의 변화가 있는 부분이 있음
3. 더 자세히 확인하기 위해, np.quantile 이용해 구간 별 확인

---

In [ ]:
index_10perent = (np.quantile(range(0, len(df.columns)-1), q=np.linspace(0, 1, 21))).astype(int)
df.describe()[index_10perent]

20분위수로 feature를 봤을 때
1. feature index가 커지거나 작아질때 기초통계량의 경향성이 보이지 않음
2. (48~194 index)와 (242~388 index)는 max 값과 std 값에서 차이 발생
3. (534~679 index) (728~825 index), (873, 922 index)에서도 max 값과 std 값에서 차이 발생

→ mean 값이 비슷한데 max값의 차이가 나는 것은, outlier 신호가 발생했다고 예상됨

→ mean 값이 비슷한데 std 차이가 나는 것은, std에 영향을 줄 정도로 많은 수의 평균 0에 가까운 노이즈 신호가 발생됐다고 예상됨

---

In [ ]:
print(df['target'].unique())

target은 0, 1, 2, 3, 4, 5, 6 으로 이루어짐

---
target (0, 1, 2, 3, 4, 5, 6) 별로 어떤 종류의 신호인지 5개씩 random sampling 해서 시각화

In [ ]:
num_samples = 5
unique_targets = np.sort(df['target'].unique())
n_unique_targets = len(unique_targets)

fig, axes = plt.subplots(n_unique_targets, num_samples, figsize=(15, 10))
for idx, target in enumerate(unique_targets):
    df_sub = df[df['target'] == target].sample(num_samples)
    for j in range(num_samples):
        ax = axes[idx, j]
        ax.plot(df_sub.iloc[j, :-1].values)
        ax.set_title(f'target: {int(target)}, sample: {j+1}')
plt.tight_layout()
plt.show()

random sample 시각화 결과 target 별로 이상 신호가 다르고 발생 구간이 다름

※ 참고: 우리가 관심을 가지는 센서 데이터의 구간이 정해져 있음

| Target | 설명                                     | 구간     |
|--------|------------------------------------------|---------|
| 0      | 정상 신호                                 |         |
| 1      | 1 point 튄 이상 신호                      | 1번 구간 |
| 2      | 1 point 튄 이상 신호                      | 2번 구간 |
| 3      | 1 point 튄 이상 신호                      | 3번 구간 |
| 4      | 100 points 이상의 흔들리는 불안정 이상 신호 | 1번 구간 |
| 5      | 100 points 이상의 흔들리는 불안정 이상 신호 | 2번 구간 |
| 6      | 100 points 이상의 흔들리는 불안정 이상 신호 | 3번 구간 |

<br>

---

정상 신호와 이상 신호 간의 클래스 비율 확인하여 불균형 있는지 확인

In [ ]:
import matplotlib.cm as cm

target_value_counts = df['target'].value_counts()
print(target_value_counts)
plt.pie(target_value_counts, labels=target_value_counts.index, explode=([0.1]*len(target_value_counts)), autopct='%1.1f%%', colors=cm.Set3.colors, startangle=90, counterclock=False)
plt.tight_layout()
plt.show()

| Target | 설명                                     | 구간     | 개수   | 비율  |
|--------|------------------------------------------|---------|--------|-------|
| 0      | 정상 신호                                 |         | 10,000 | 73.5% |
| 1      | 1 point 튄 이상 신호                      | 1번 구간 | 600    | 4.4%  |
| 2      | 1 point 튄 이상 신호                      | 2번 구간 | 600    | 4.4%  |
| 3      | 1 point 튄 이상 신호                      | 3번 구간 | 600    | 4.4%  |
| 4      | 100 points 이상의 흔들리는 불안정 이상 신호 | 1번 구간 | 600    | 4.4%  |
| 5      | 100 points 이상의 흔들리는 불안정 이상 신호 | 2번 구간 | 600    | 4.4%  |
| 6      | 100 points 이상의 흔들리는 불안정 이상 신호 | 3번 구간 | 600    | 4.4%  |

정상 신호 대 이상 신호 비율 - 73.5% : 26.5%
- 정상 신호와 이상 신호 간 클래스 비율이 적절함

---
신호 한 개의 데이터에서 이상 신호 데이터 비율 확인 (비정상/정상)

In [ ]:
target_values = [1, 2, 3, 4, 5, 6]
n = 1

np.random.seed(0)
plt.figure(figsize=(15, 5))
for i, target_value in enumerate(target_values, start=1):
    target_df = df[df['target'] == target_value].iloc[:, :-1]
    random_indices = np.random.randint(0, min(len(target_df), 600), n)
    target_indices = target_df.iloc[random_indices]

    plt.subplot(2, 3, i)
    for index, row in enumerate(target_indices.values):
        plt.plot(row, label=f'index {random_indices[index]}')
    plt.legend(loc='best')
    plt.title(f'target {target_value}')
plt.tight_layout()
plt.show()

target 별로 데이터 1개씩 sampling 해서 시각화 (plt.xlim() 이용해 확대해서 확인, 길어서 생략)
- target 1, 2, 3 구간: 이것만 봐서 모름, 중복해서 여러개 겹친 후 판단
- target 4 구간: 200 ~ 400 (200 points)
- target 5 구간: 500 ~ 700 (200 points)
- target 6 구간: 850 ~ 940 (90 points)

---

In [ ]:
target_values = [1, 2, 3, 4, 5, 6]
n = 30

vertical_lines = {
    1: [200, 400],
    2: [500, 700],
    3: [850, 940],
    4: [200, 400],
    5: [500, 700],
    6: [850, 940]
}

np.random.seed(0)
plt.figure(figsize=(15, 8))
for i, target_value in enumerate(target_values, start=1):
    target_df = df[df['target'] == target_value].iloc[:, :-1]
    random_indices = np.random.randint(0, 600, n)
    target_indices = target_df.iloc[random_indices]

    plt.subplot(2, 3, i)
    for index, row in enumerate(target_indices.values):
        plt.plot(row)
    for line in vertical_lines[target_value]:
        plt.axvline(x=line, color='r', linestyle='--')
    plt.title(f'target {target_value}')
plt.tight_layout()
plt.show()

target 별로 데이터 30개씩 sampling 해서 시각화
- target 1 구간: 200 ~ 400 (1 points)
- target 2 구간: 500 ~ 700 (1 points)
- target 3 구간: 850 ~ 940 (1 points)
- target 4 구간: 200 ~ 400 (200 points)
- target 5 구간: 500 ~ 700 (200 points)
- target 6 구간: 850 ~ 940 (90 points)

---

위 시각화 결과를 통해 다음과 같은 패턴을 확인할 수 있었습니다.

| Target | 설명                                     | index   | 개수   | 비율  |
|--------|------------------------------------------|---------|--------|-------|
| 0      | 정상 신호                                 |         | 10,000 | 73.5% |
| 1      | 1 point 튄 이상 신호                      | 200~400 | 600    | 4.4%  |
| 2      | 1 point 튄 이상 신호                      | 500~700 | 600    | 4.4%  |
| 3      | 1 point 튄 이상 신호                      | 850~940 | 600    | 4.4%  |
| 4      | 200 points 흔들리는 불안정 이상 신호       | 200~400 | 600    | 4.4%  |
| 5      | 200 points 이상의 흔들리는 불안정 이상 신호 | 500~700 | 600    | 4.4%  |
| 6      | 90 points 이상의 흔들리는 불안정 이상 신호  | 850~940 | 600    | 4.4%  |

이 내용을 기반으로 Feature scaling 진행 후, 해당 데이터에 적합한 머신러닝 모델을 개발하겠습니다.

---
## **모델**

- 설명 불가능한 딥러닝 모델 보다는 설명할 수 있는 머신러닝 모델을 우선 고려
- target 데이터가 있기 때문에 분류해주는 머신러닝 모델 평가 (강사님이 다 돌려보래서 배운 분류 모델 전부 돌릴 예정)

---
머신러닝 모델 학습을 위한 Feature scaling

**Standardization** 선정

선정 근거

- Standardization
   1. 평균과 표준편차를 이용하기 떄문에, 이상 신호 값이 표준편차에 영향을 줄 정도로 많지 않는 이상 데이터 전체에 미치는 영향이 Normalization에 비해 상대적으로 적음

- Normalization
   1. 이상 신호 값이 엄청나게 크면 데이터가 전체적으로 한쪽으로 쏠림 → 따라서 정상 값들이 작은 구간 내로 몰릴 것으로 예상됨

In [ ]:
def print_result_classifier(y, pred):
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    from sklearn.metrics import confusion_matrix, classification_report
    accuracy = accuracy_score(y, pred)
    precision = precision_score(y, pred, average='macro')
    recall = recall_score(y, pred, average='macro')
    f1 = f1_score(y, pred, average='macro')
    print(f'accuracy: {accuracy:.6f}, precision: {precision:.6f}, recall: {recall:.6f}, f1_score: {f1:.6f}')
    print(confusion_matrix(y, pred))
    print(classification_report(y, pred))
    
def print_result_regressor(y, pred):
    from sklearn.metrics import mean_squared_error, r2_score
    print(f'mse: {mean_squared_error(y, pred):.6f}, r2_score: {r2_score(y, pred):.6f}')

In [ ]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [ ]:
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size = 0.2/0.8, random_state=0)

print(f'X_train: {X_train.shape}, X_val: {X_val.shape}, X_test: {X_test.shape}')
print(f'y_train: {y_train.shape}, y_val: {y_val.shape}, y_test: {y_test.shape}')

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

---
### 혼자 Test 해보기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_pickle('./data/signal/data_1.pkl')

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size = 0.2/0.8, random_state=0)

print(f'X_train: {X_train.shape}, X_val: {X_val.shape}, X_test: {X_test.shape}')
print(f'y_train: {y_train.shape}, y_val: {y_val.shape}, y_test: {y_test.shape}')

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# 1초
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=2)
clf.fit(X_train_scaled, y_train)
print_result_classifier(y_test, clf.predict(X_test_scaled))

In [ ]:
# 1초
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty='l2')
clf.fit(X_train_scaled, y_train)
print_result_classifier(y_test, clf.predict(X_test_scaled))

In [ ]:
# 1초
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train_scaled, y_train)
print_result_classifier(y_test, clf.predict(X_test_scaled))

In [ ]:
# 5분
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train_scaled, y_train)
print_result_classifier(y_test, clf.predict(X_test_scaled))

In [ ]:
# 1분 30초
from sklearn.svm import SVC
clf = SVC(kernel='linear')
clf.fit(X_train_scaled, y_train)
print_result_classifier(y_test, clf.predict(X_test_scaled))

In [ ]:
# 1분 30초
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier

clf = VotingClassifier(
    estimators=[
        ('logistic_regression', LogisticRegression()),
        ('svc', SVC(kernel='linear')),
        ('gaussianNB', GaussianNB())
    ],
    voting='hard',
    weights=[1, 1, 1]
)
clf.fit(X_train_scaled, y_train)
print_result_classifier(y_test, clf.predict(X_test_scaled))

In [ ]:
# 5초
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2,
                             random_state=0)
clf.fit(X_train_scaled, y_train)
print_result_classifier(y_test, clf.predict(X_test_scaled))

In [ ]:
# 4초
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(estimator=GaussianNB(),
                        n_estimators=10,
                        random_state=0)
clf.fit(X_train_scaled, y_train)
print_result_classifier(y_test, clf.predict(X_test_scaled))

In [ ]:
# 1분
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(random_state=0)
clf.fit(X_train_scaled, y_train)
print_result_classifier(y_test, clf.predict(X_test_scaled))

In [ ]:
# 18분
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(max_depth=2, learning_rate=0.1, random_state=0)
clf.fit(X_train_scaled, y_train)
print_result_classifier(y_test, clf.predict(X_test_scaled))

In [ ]:
# 2분 30초
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
clf1 = SVC(kernel='linear', random_state=1)
clf2 = GaussianNB()
clf = StackingClassifier(
    estimators=[
        ('svm', clf1),
        ('gnb', clf2)
    ],
    final_estimator=LogisticRegression())
clf.fit(X_train_scaled, y_train)
print_result_classifier(y_test, clf.predict(X_test_scaled))

In [ ]:
# 3초
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
clf = LinearDiscriminantAnalysis()
clf.fit(X_train_scaled, y_train)
print_result_classifier(y_test, clf.predict(X_test_scaled))

from sklearn.ensemble import RandomForestClassifier
X_tn_lda = clf.transform(X_train_scaled)
X_te_lda = clf.transform(X_test_scaled)
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_tn_lda, y_train)
print_result_classifier(y_test, clf.predict(X_te_lda))

In [ ]:
# 4초
from sklearn.cluster import KMeans
clf = KMeans(n_clusters=7, init='random', n_init=10, max_iter=100, random_state=0)
clf.fit(X)
pred = clf.labels_

In [ ]:
pred_val_counts = pd.DataFrame(pred).value_counts()
plt.pie(x=pred_val_counts, labels=pred_val_counts.index, autopct='%1f%%')
plt.show()

In [ ]:
# 14초
from sklearn.cluster import DBSCAN
clf = DBSCAN(eps=0.2)
clf.fit(X)
pred = clf.labels_

In [ ]:
pred_val_counts = pd.DataFrame(pred).value_counts()
plt.pie(x=pred_val_counts, labels=pred_val_counts.index, autopct='%1f%%')
plt.show()

In [ ]:
# 35초
from sklearn.mixture import GaussianMixture
clf = GaussianMixture(n_components=7, random_state=0)
clf.fit(X)
pred = clf.predict(X)

In [ ]:
pred_val_counts = pd.DataFrame(pred).value_counts()
plt.pie(x=pred_val_counts, labels=pred_val_counts.index, autopct='%1f%%')
plt.show()

In [ ]:
# 2초
from sklearn.svm import OneClassSVM
clf = OneClassSVM(gamma='auto', nu=0.01)
clf.fit(X)
pred = clf.predict(X)

In [ ]:
pred_val_counts = pd.DataFrame(pred).value_counts()
plt.pie(x=pred_val_counts, labels=pred_val_counts.index, autopct='%1f%%')
plt.show()

In [ ]:
# 4초
from sklearn.neighbors import LocalOutlierFactor
clf = LocalOutlierFactor(n_neighbors=7)
pred = clf.fit_predict(X)

In [ ]:
pred_val_counts = pd.DataFrame(pred).value_counts()
plt.pie(x=pred_val_counts, labels=pred_val_counts.index, autopct='%1f%%')
plt.show()

In [ ]:
# 1초
from sklearn.ensemble import IsolationForest
clf = IsolationForest(random_state=0)
clf.fit(X)
pred = clf.predict(X)

In [ ]:
pred_val_counts = pd.DataFrame(pred).value_counts()
plt.pie(x=pred_val_counts, labels=pred_val_counts.index, autopct='%1f%%')
plt.show()

In [ ]:
from sklearn.decomposition import PCA
clf = PCA(n_components=7)
clf.fit(X_train_scaled, y_train)
X_tn_pca = clf.transform(X_train_scaled)
X_te_pca = clf.transform(X_test_scaled)

### 혼자 Test 해보기 끝
---

---
## **결론**

### <결론 가이드>
- 처음 정의한 문제 해결
- 문제와 관련된 새로운 사실
- 기존 업무 효율화
- 데이터셋에 대한 인사이트
- 등등

여러 관점에서 프로젝트의 결과를 정리하여 최종 결론을 논리적으로 서술

추천 시스템을 통해 구매 전환율이 예상보다 X% 향상될 것으로 예측합니다.

추가 데이터 수집 및 다른 알고리즘의 적용 가능성에 대해서도 탐색합니다.

더 정밀한 타겟팅과 개인화된 추천을 위해, 향후에는 고급 기법들을 도입할 계획입니다. 예를 들어, 딥러닝 기반의 추천 시스템이나, 시간에 따라 변화하는 고객의 선호를 반영할 수 있는 동적 모델링 방법들입니다.

이 프로젝트를 통해 얻은 인사이트와 모델은 실제 업무 프로세스에 통합될 예정입니다. 예를 들어, 마케팅 캠페인의 타겟팅 전략 수립, 재고 관리 및 물류 계획의 최적화, 신제품 개발의 방향성 결정 등 다양한 분야에서 활용될 수 있습니다.
또한, 프로젝트 결과는 내부 리포트와 워크숍을 통해 공유되어, 조직 전반의 데이터 기반 의사결정 문화를 강화하는 데 기여할 것입니다.